# Almacenamiento de datos

## Introducción

Vamos a generar una BBDD Relacional para el almacenamiento de los registros que hemos obtenido hasta ahora.

En este momento del proyecto (mayo/23) prepararemos una base de datos con 2 Tablas y posteriormente generaremos una tercera tabla con las variables que habremos generado en el notebook de Engineering para generar #tags. 

## Librerías

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import sqlite3
# import mysql.connector


In [2]:
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online_webscrapping\\src'

## Estructuras de la BBDD

![Estructura_tablas](../Resources/image/Relacion_tablas.jpg)

## 1. Construyendo la BBDD

### a.- Creando la estructura

In [3]:
# Conectamos con la base de datos
conn = sqlite3.connect("Data/online_shop.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
cursor = conn.cursor()

### Funciones para obtener datos y ver en un dataframe

In [4]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    cursor.execute(query)

    # Almacena los datos de la query 
    ans = cursor.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor.description]

    return pd.DataFrame(ans,columns=names)

In [5]:
res = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

PRODUCT
COMMENT


#### PRODUCT

In [7]:
query = '''
CREATE TABLE PRODUCT  (
    ID VARCHAR (3),
    PRODUCT VARCHAR (100),
    SLOGAN VARCHAR (100),
    DESCRIPTION VARCHAR (250),
    CHARACTERISTICS VARCHAR (250),
    LISTA_URL VARCHAR (50),
    REGULAR_PRICE INT (3),
    DISCOUNT_PRICE INT (3),
    PRIMARY KEY (ID)
)

'''


cursor.execute(query)

#### COMMENTS

In [8]:
query = '''
CREATE TABLE COMMENT  (
    ID_COMMENT VARCHAR (5),
    ID_PRODUCT VARCHAR (3),
    DATE DATETIME (100),
    RATIO INT (2),
    USERS VARCHAR (50),
    COMMENT VARCHAR (250), 
    PRIMARY KEY (ID_COMMENT),
    FOREIGN KEY (ID_PRODUCT) REFERENCES PRODUCT (ID)
)

'''

cursor.execute(query)

### b.- Cargando los datos dentro de la BBDD

In [6]:
df_products=pd.read_csv(r'Data\productos.csv')
print("Long. dataframe",len(df_products.columns))
df_products.head()

Long. dataframe 8


,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,Desliz! Lubricante íntimo de agua 100ml,NaN,"Algunos lubricantes son un poco densos, otros ...",Bote de 100ml de venta exclusiva en amantis.ne...,https://www.amantis.net/desliz-lubricante-inti...,9.99,7.99
1,1,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,Vuelve nuestro vibrador de doble estimulación ...,"Medidas: 19cm (11cm insertables) y 3,3cm/ 2,2c...",https://www.amantis.net/tobogane-hot-rabbit-el...,89.99,39.99
2,2,BALLENATO,tu vibrador a distancia con aleta móvil y sume...,De las profundidades más húmedas llega BALLENA...,Mando a distancia.Peso 62 gramos.Silicona médi...,https://www.amantis.net/ballenato-tu-vibrador-...,99.99,49.99
3,3,TANDEM 2 flex,vibrador doble flexible con mando,Tanto si ya eras fan de nuestro querido TANDEM...,10 patrones de vibración diferentes y tres niv...,https://www.amantis.net/tandem-2-flex-vibrador...,109.99,49.99
4,4,FOXY,Succionador con mango vibrador y orejitas,"Los succionadores van evolucionando, vamos des...",Vibrador con orejitas y succión simultáneaMate...,https://www.amantis.net/foxy-vibrador-succiona...,59.99,39.99


In [20]:
# df_comments=pd.read_csv(r'Data\comentarios_scrape.csv',index_col=False)
df_comments=pd.read_csv(r'Data\comentarios.csv',index_col=False)
df_comments.rename(columns={"ID":"ID_PRODUCT","Unnamed: 0":"ID_COMMENT"},inplace=True)
# df_comments['DATE'] = pd.to_datetime(df_comments.iloc[:,-3:])
# df_comments=df_comments.iloc[:,:-3].head()                                                  # Eliminamos las columnas temporales creadas
print("Long. dataframe",len(df_comments.columns))
df_comments.head()

Long. dataframe 6


,ID_COMMENT,ID_PRODUCT,DATE,RATIO,USERS,COMMENT
0,0,0,2023-05-18,5,David,"Muy recomendable. El bote cunde, hidrata muy b..."
1,1,0,2023-04-12,5,Ferran,Es la primera vez que compro este lubricante y...
2,2,0,2023-04-10,5,Sergio,He probado varios tipos de lubricante y este d...
3,3,0,2023-01-23,5,Jaime,Perfecto para embadurnarte el pincelin y meter...
4,4,0,2023-01-11,5,YASMINA,"Buen producto, tanto para el cuerpo a cuerpo c..."


In [21]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10659 entries, 0 to 10658
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID_COMMENT  10659 non-null  int64 
 1   ID_PRODUCT  10659 non-null  int64 
 2   DATE        10659 non-null  object
 3   RATIO       10659 non-null  int64 
 4   USERS       10648 non-null  object
 5   COMMENT     10659 non-null  object
dtypes: int64(3), object(3)
memory usage: 499.8+ KB


In [22]:
lista_products= df_products.values.tolist()                                                #   Pasamos a lista el dataframe

cursor.executemany("INSERT INTO PRODUCT VALUES (?,?,?,?,?,?,?,?)", lista_products)           #   Salvamos directamente todos los datos en la tabla PRODUCTS

lista_comments= df_comments.values.tolist()                                                #   Pasamos a lista el dataframe

cursor.executemany("INSERT INTO COMMENT VALUES (?,?,?,?,?,?)", lista_comments)                 #   Salvamos directamente todos los datos en la tabla COMMENTS

In [23]:
conn.commit()
cursor.close()
conn.close()

## Cerramos la BBDD.

In [7]:
conn.commit()
cursor.close()
conn.close()